# Command-Line Interface

> Command-line tools for barcode extraction and processing

## Introduction
This notebook implements the command-line interface (CLI) for BarcodeSeqKit, allowing users to easily extract barcodes from BAM and FASTQ files without writing Python code.

In [ ]:
#| default_exp cli

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import sys
import logging
import argparse
from typing import List, Optional, Dict, Any, Tuple

from BarcodeSeqKit.core import (
    BarcodeConfig, 
    BarcodeLocationType,
    BarcodeSeqKitConfig,
    FileFormat
)
from BarcodeSeqKit.bam_processing import extract_barcodes_from_bam
from BarcodeSeqKit.fastq_processing import extract_barcodes_from_fastq

## Command-Line Argument Parser
Let's define the argument parser for the command-line interface.

In [ ]:
#| export
def add_arguments(parser: argparse.ArgumentParser) -> argparse.ArgumentParser:
    """Add arguments to the argument parser.
    
    Args:
        parser: ArgumentParser to add arguments to
        
    Returns:
        Updated ArgumentParser
    """
    # Input files (required)
    input_group = parser.add_argument_group('Input options')
    
    # Input can be specified in different ways
    input_mutex = input_group.add_mutually_exclusive_group(required=True)
    
    input_mutex.add_argument(
        '--bam',
        help='Input BAM file'
    )
    
    input_mutex.add_argument(
        '--fastq1',
        help='Input FASTQ file (read 1 for paired-end data)'
    )
    
    input_mutex.add_argument(
        '--fastq-dir',
        help='Directory containing paired FASTQ files'
    )
    
    # When specifying fastq1, fastq2 is optional
    input_group.add_argument(
        '--fastq2',
        help='Input FASTQ file (read 2 for paired-end data, required if --fastq1 is used)'
    )
    
    # Barcode configuration (required)
    barcode_group = parser.add_argument_group('Barcode options')
    
    barcode_mutex = barcode_group.add_mutually_exclusive_group(required=True)
    
    barcode_mutex.add_argument(
        '--barcode-config',
        help='YAML file containing barcode configurations'
    )
    
    barcode_mutex.add_argument(
        '--barcode',
        help='Single barcode sequence (no specific location, will generate barcode_orientFR and barcode_orientRC outputs)'
    )
    
    barcode_mutex.add_argument(
        '--barcode5',
        help="5' barcode sequence (will generate barcode5_orientFR and barcode5_orientRC outputs)"
    )
    
    barcode_group.add_argument(
        '--barcode3',
        help="3' barcode sequence (will generate barcode3_orientFR and barcode3_orientRC outputs, can be used with --barcode5)"
    )
    
    barcode_group.add_argument(
        '--max-mismatches',
        type=int,
        default=0,
        help='Maximum number of mismatches allowed in barcode detection (default: 0)'
    )
    
    # Output options
    output_group = parser.add_argument_group('Output options')
    
    output_group.add_argument(
        '--output-prefix',
        required=True,
        help='Prefix for output files'
    )
    
    output_group.add_argument(
        '--output-dir',
        default='.',
        help='Directory for output files (default: current directory)'
    )
    
    output_group.add_argument(
        '--merge-orientations',
        action='store_true',
        help='Merge forward and reverse complement orientations'
    )
    
    # Processing options
    processing_group = parser.add_argument_group('Processing options')
    
    processing_group.add_argument(
        '--search-both-reads',
        action='store_true',
        help='Search for barcodes in both reads of paired-end data (FASTQ only)'
    )
    
    processing_group.add_argument(
        '--search-softclipped',
        action='store_true',
        help='Search for barcodes in softclipped read region (BAM only)'
    )
    
    processing_group.add_argument(
        '--no-compress',
        action='store_true',
        help='Do not compress output FASTQ files (FASTQ only)'
    )
    
    # Logging options
    logging_group = parser.add_argument_group('Logging options')
    
    logging_group.add_argument(
        '--verbose',
        action='store_true',
        help='Enable verbose logging'
    )
    
    logging_group.add_argument(
        '--log-file',
        help='Log file path'
    )
    
    return parser

## Configuration Setup
Now let's implement functions to create a BarcodeSeqKitConfig from command-line arguments.

In [ ]:
#| export
def create_config_from_args(args: argparse.Namespace) -> Tuple[BarcodeSeqKitConfig, List[str]]:
    """Create a BarcodeSeqKitConfig from command-line arguments.
    
    Args:
        args: Parsed command-line arguments
        
    Returns:
        Tuple of (config, input_files)
        
    Raises:
        ValueError: If required arguments are missing
    """
    # Determine input files
    input_files = []
    
    if args.bam:
        input_files = [args.bam]
    elif args.fastq1:
        if args.fastq2:
            input_files = [args.fastq1, args.fastq2]
        else:
            raise ValueError("--fastq2 is required when using --fastq1")
    elif args.fastq_dir:
        input_files = [args.fastq_dir]
    
    # Load or create barcode configurations
    barcodes = []
    
    if args.barcode_config:
        # Load from YAML file
        config = BarcodeSeqKitConfig.load_yaml(args.barcode_config)
        barcodes = config.barcodes
    else:
        # Create from command-line arguments
        if args.barcode:
            # Single barcode mode (no specific location)
            barcodes.append(BarcodeConfig(
                sequence=args.barcode,
                location=BarcodeLocationType.UNKNOWN,
                name="generic",
                description="Generic barcode from command line"
            ))
        elif args.barcode5:
            barcodes.append(BarcodeConfig(
                sequence=args.barcode5,
                location=BarcodeLocationType.FIVE_PRIME,
                name="5",
                description="5' barcode from command line"
            ))
            
            if args.barcode3:
                barcodes.append(BarcodeConfig(
                    sequence=args.barcode3,
                    location=BarcodeLocationType.THREE_PRIME,
                    name="3",
                    description="3' barcode from command line"
                ))
        elif args.barcode3:
            # If only barcode3 is provided without barcode5, treat as single barcode
            barcodes.append(BarcodeConfig(
                sequence=args.barcode3,
                location=BarcodeLocationType.UNKNOWN,
                name="generic",
                description="Generic barcode from command line (provided as barcode3)"
            ))
    
    # Create the configuration object
    config = BarcodeSeqKitConfig(
        barcodes=barcodes,
        output_prefix=args.output_prefix,
        output_dir=args.output_dir,
        merge_orientations=args.merge_orientations,
        verbose=args.verbose,
        log_file=args.log_file,
        max_mismatches=args.max_mismatches,
        search_softclipped=args.search_softclipped,
        search_both_reads=args.search_both_reads,
        compress_output=not args.no_compress
    )
    
    return config, input_files

## Main CLI Runner
Now let's implement the main CLI runner function.

In [ ]:
#| export
def run_cli(args: Optional[List[str]] = None) -> int:
    """Run the command-line interface.
    
    Args:
        args: Command-line arguments (if None, use sys.argv)
        
    Returns:
        Exit code (0 for success, non-zero for error)
    """
    parser = argparse.ArgumentParser(
        description="BarcodeSeqKit: A toolkit for barcode extraction from sequencing data",
        formatter_class=argparse.RawDescriptionHelpFormatter
    )
    
    parser = add_arguments(parser)
    
    try:
        # Parse arguments
        parsed_args = parser.parse_args(args)
        
        # Create configuration
        config, input_files = create_config_from_args(parsed_args)
        
        # Save the configuration for reproducibility
        config_path = os.path.join(parsed_args.output_dir, f"{parsed_args.output_prefix}_config.yaml")
        config.save_yaml(config_path)
        config.logger.info(f"Saved configuration to {config_path}")
        
        # Determine input file format
        file_format = FileFormat.detect_format(input_files[0])
        
        # Process files based on format
        if file_format == FileFormat.BAM:
            config.logger.info(f"Processing BAM file: {input_files[0]}")
            extract_barcodes_from_bam(input_files[0], config)
        elif file_format == FileFormat.FASTQ:
            config.logger.info(f"Processing FASTQ files: {', '.join(input_files)}")
            extract_barcodes_from_fastq(input_files, config)
        else:
            raise ValueError(f"Unsupported file format: {file_format}")
        
        config.logger.info("Processing complete")
        return 0
    
    except Exception as e:
        print(f"Error: {str(e)}")
        if hasattr(parsed_args, 'verbose') and parsed_args.verbose:
            import traceback
            traceback.print_exc()
        return 1

In [ ]:
#| export
def main() -> int:
    """Main entry point for the command-line script."""
    return run_cli()

In [ ]:
#| export
if __name__ == "__main__":
    sys.exit(main())

## Setting Up the Command-Line Script
To make BarcodeSeqKit installable as a command-line tool, we need to create an entry point in the package's setup configuration.

In your `settings.ini` file, add the following entry point configuration:

```ini
[entry_points]
console_scripts = 
    barcodeseqkit = BarcodeSeqKit.cli:main
```

This will create a command-line script called `barcodeseqkit` that calls the `main()` function in this module.

## Example Usage
Here are some examples of how to use the command-line interface:

### Extract barcodes from a BAM file

```bash
barcodeseqkit --bam sample.bam --barcode ACTGACTG --output-prefix sample_barcoded --output-dir ./output
```

### Extract barcodes from paired FASTQ files

```bash
barcodeseqkit --fastq1 read1.fastq.gz --fastq2 read2.fastq.gz --barcode5 ACTGACTG --output-prefix sample_barcoded --output-dir ./output
```

### Extract 5' and 3' barcodes from a directory containing FASTQ files

```bash
barcodeseqkit --fastq-dir ./fastq_dir --barcode5 ACTGACTG --barcode3 GTCAGTCA --output-prefix sample_barcoded --output-dir ./output --search-both-reads
```

### Use a barcode configuration file

```bash
barcodeseqkit --bam sample.bam --barcode-config barcodes.yaml --output-prefix sample_barcoded --output-dir ./output
```

## Conclusion
This notebook implements a command-line interface for BarcodeSeqKit, making it easy to use the library's functionality without writing Python code. The CLI provides access to all the major features of the library, including barcode extraction from BAM and FASTQ files, customization options for searching and output, and comprehensive logging.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()